# 설치 파일

## 데이터 다운로드

groove zip파일 gdown으로 다운로드

In [2]:
!gdown 1xyP9fEB6hgzah6D8c3xjXu7mpzhqssOE

Downloading...
From: https://drive.google.com/uc?id=1xyP9fEB6hgzah6D8c3xjXu7mpzhqssOE
To: /content/groove-v1.0.0-midionly.zip
100% 3.26M/3.26M [00:00<00:00, 252MB/s]


## magenta 패키지 설치

|erorr note| dependency 이슈: magenta 2.1.0 버전 설치

In [3]:
!pip install magenta==2.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 15.7 MB/s 
     |████████████████████████████████| 69 kB 9.3 MB/s 
     |████████████████████████████████| 90 kB 12.0 MB/s 
     |████████████████████████████████| 5.6 MB 58.3 MB/s 
     |████████████████████████████████| 209 kB 62.4 MB/s 
     |████████████████████████████████| 268 kB 73.0 MB/s 
     |████████████████████████████████| 2.3 MB 73.2 MB/s 
     |████████████████████████████████| 3.6 MB 72.6 MB/s 
     |████████████████████████████████| 14.5 MB 69.9 MB/s 
     |████████████████████████████████| 352 kB 80.5 MB/s 
     |████████████████████████████████| 1.4 MB 69.5 MB/s 
     |████████████████████████████████| 204 kB 10.2 MB/s 
     |████████████████████████████████| 526 kB 61.8 MB/s 
     |████████████████████████████████| 2.5 MB 65.8 MB/s 
     |████████████████████████████████| 278 kB 76.0 MB/s 
     |███████████████████████████

# 전처리

전처리 이유
- midi데이터를 학습하기위해, 백터화 작업 필요요.
- Magenta의 convert_directory 이용해 전처리
- zip파일로 다운받고 zipfile 라이브러리로 압축 해제 후, 백터화
- 백터화하여 TFrecord로 저장

TFRecord 파일은 tensorflow의 학습 데이터등을 저장하기 위한 이진 데이터 포맷으로, 구글의 Protocol Buffer 포맷으로 데이터를 파일에 직렬화하여 저장

필요성
- 이미지 데이터와 같은 데이터는 메타데이터와 레이블이 별도의 파일로 저장되어 있기 때문에, 각각 읽어들여야해서 코드가 복잡해짐.TFrecord 파일을 이용하면 훨씬 간단하게 해결 가능.
- midi 포맷맷으로 읽어서 매번 디코딩을 진행하면 학습단계에서 데이터를 읽을때 성능저하가 발생. 이를 해결하기 위해 TFrecord 파일을 사용 가능.

In [4]:
# 필요 module
import tensorflow as tf
import numpy as np
import pathlib # 경로를 객체로 처리하기위해
import zipfile # zip파일 압축 해제를 위해
import os
import sys
import time
import pandas as pd
import IPython
import collections
import note_seq # 시퀀스 midi화

import magenta.music as mm
from magenta.common import merge_hparams
from magenta.contrib import training as contrib_training
from magenta.models.music_vae import MusicVAE
from magenta.models.music_vae import lstm_models
from magenta.models.music_vae import data
from magenta.scripts.convert_dir_to_note_sequences import convert_directory # 전처리
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel # 훈련 모델
import tensorflow.compat.v1 as tf
import tf_slim # TF-Slim은 저수준의 텐서플로우 API를 간편하게 사용할 수 있는 고수준 경량 API

/usr/local/lib/python3.8/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


In [5]:
# zip파일 압축해제
zipfile.ZipFile('./groove-v1.0.0-midionly.zip').extractall()

In [6]:
# TFRecord 저장할 디렉토리 생성성
os.mkdir("data_TFRecord")

In [7]:
# 경로지정
data_root= './groove' # 데이터 경로
csv_file = './groove/info.csv' # midi 파일의 session, id, bpm 정보를 담고있음.
tfrec_root = './data_TFRecord/music.tfrecord'# TFRecord 파일 경로를 지정

In [8]:
# 데이터 확인인
df = pd.read_csv('./groove/info.csv')
df = pd.DataFrame(df)
df.head(3)

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,test


## 전처리 진행(TFRecord화)

In [9]:
convert_directory(data_root,tfrec_root,recursive=True) # 전처리 함수함수

# 모델 학습

## Config 정의

- magenta/blob/main/magenta/models/music_vae/configs.py 수정
- 모델과 함께 전체 config를 수정
- TFRecord 경로지정

### 모델 구조

**Encoder** : BidirectionalLSTM(양방향 LSTM)
- 2계층 양방향 LSTM
 - 역방향(미래에서 과거) 또는 정방향(과거에서 미래) 두 양방향으로 시퀀스 정보를 갖도록 만듬.  
 - BidirectionalLSTM 을 사용하면 미래와 과거의 정보를 보전하기위해 양방향으로 입력흐름을 만들수 있음.
- NLP모델인 BERT에서도 사용  

**Decoder** : Hierarchical Decoder(CategoricalLstm)  
- Simple RNN 사용 하면 기울기소실로 성능저하
 - 하위 시퀀스 간의 복잡한 종속성을 갖는 계층구조 생성 프로세스를 모델링 하기위해 다양한 시간단계에 걸쳐, 잠재 확률 변수를 사용한 신경망 기반 생성 아키텍처
- NLP모델에서 많이사용

In [49]:
# configs.py 일부 발췌

class Config(collections.namedtuple(
    'Config',
    ['model', 'hparams', 'note_sequence_augmenter', 'data_converter',
     'train_examples_path', 'eval_examples_path', 'tfds_name'])):

    def values(self):
        return self._asdict()

Config.__new__.__defaults__ = (None,) * len(Config._fields)


def update_config(config, update_dict):
    config_dict = config.values()
    config_dict.update(update_dict)
    return Config(**config_dict)


CONFIG_MAP = {}


HParams = contrib_training.HParams

# 모델 config configs.py에서 4마디 드럼 파일을 선택 위해 'cat-drums_2bar_small'내용을 인용

# 모델구조
CONFIG_MAP['cat-drums_4bar_small'] = Config(
    model=MusicVAE(lstm_models.BidirectionalLstmEncoder(),  # BidirectionalLstm Encoder 사용
                   lstm_models.CategoricalLstmDecoder()),  # Hierarchical Decoder 사용
    hparams=merge_hparams(
        lstm_models.get_default_hparams(),
        HParams(
            batch_size=512,  # 데이터 배치사이즈
            max_seq_len=16 * 4,  # 4마디 길이지정
            z_size=256, # 잠재백터 사이즈, 잠재 공간(latent space) 설정정
            enc_rnn_size=[512], # 인코더 순환 사이즈지정
            dec_rnn_size=[256, 256], # 디코더 순환사이즈 지정
            free_bits=48,
            max_beta=0.2,
            sampling_schedule='inverse_sigmoid',
            sampling_rate=1000,
        )),
    note_sequence_augmenter=None,
    data_converter=data.DrumsConverter( # 드럼 파일만 선택
        max_bars=100,
        slice_bars=4, # 4 마디 단위로 시퀀스 나눔
        steps_per_quarter=4,
        roll_input=True,),
    train_examples_path='./data_TFRecord/music.tfrecord', # 데이터 경로 설정
)

다음 코드들 통해 전처리 해야하지만 구현 실패(생략하고 진행)

In [39]:
cd ./magenta

/content/magenta


In [44]:
CONFIG = 'cat-drums_4bar_small'

!python -m magenta.models.music_vae.preprocess_tfrecord \
--input_tfrecord=/content/data_TFRecord/music.tfrecord \
--output_tfrecord=/content/data_TFRecord/train-$CONFIG.tfrecord \
--output_shards=10 \
--config=$CONFIG \
--is_drum=True \
--drums_only=True \
--alsologtostderr

/usr/local/lib/python3.8/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)
Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/content/magenta/magenta/models/music_vae/preprocess_tfrecord.py", line 262, in <module>
    app.run(main)
  File "/usr/local/lib/python3.8/dist-packages/absl/app.py", line 308, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.8/dist-packages/absl/app.py", line 254, in _run_main
    sys.exit(main(argv))
  File "/content/magenta/magenta/models/music_vae/preprocess_tfrecord.py", line 257, in main
    run_pipel

In [47]:
cd /content/

/content


## Train Model 정의

In [50]:
#============ License=====================================================
# Copyright 2022 The Magenta Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#=======================================================================

#=== MusicVAE train script===
#=== magenta/models/music_vae/music_vae_train.py ===

def _trial_summary(hparams, examples_path, output_dir):
     # 텐서보드 summary 텍스트

    examples_path_summary = tf.summary.text(
        'examples_path', tf.constant(examples_path, name='examples_path'),
        collections=[])

    hparams_dict = hparams.values()

    # 하이퍼 파라미터
    header = '| Key | Value |\n| :--- | :--- |\n'
    keys = sorted(hparams_dict.keys())
    lines = ['| %s | %s |' % (key, str(hparams_dict[key])) for key in keys]
    hparams_table = header + '\n'.join(lines) + '\n'

    hparam_summary = tf.summary.text(
        'hparams', tf.constant(hparams_table, name='hparams'), collections=[])

    with tf.Session() as sess:
        writer = tf.summary.FileWriter(output_dir, graph=sess.graph)
        writer.add_summary(examples_path_summary.eval())
        writer.add_summary(hparam_summary.eval())
        writer.close()


def _get_input_tensors(dataset, config):
    # 데이터로부터 텐서 입력
    batch_size = config.hparams.batch_size
    iterator = tf.data.make_one_shot_iterator(dataset)
    (input_sequence, output_sequence, control_sequence,sequence_length) = iterator.get_next()
    input_sequence.set_shape(
        [batch_size, None, config.data_converter.input_depth])
    output_sequence.set_shape(
        [batch_size, None, config.data_converter.output_depth])
    
    if not config.data_converter.control_depth:
        control_sequence = None
    
    else:
        control_sequence.set_shape(
            [batch_size, None, config.data_converter.control_depth])
    sequence_length.set_shape([batch_size] + sequence_length.shape[1:].as_list())
        
    return {
        'input_sequence': input_sequence,
        'output_sequence': output_sequence,
        'control_sequence': control_sequence,
        'sequence_length': sequence_length
    }

# 훈련 체크포인트, 시간 설정
def train(train_dir,
          config,
          dataset_fn,
          checkpoints_to_keep=5,
          keep_checkpoint_every_n_hours=1,
          num_steps=None,
          master='',
          num_sync_workers=0,
          num_ps_tasks=0,
          task=0):

    # train loop
    tf.gfile.MakeDirs(train_dir)
    is_chief = (task == 0)

    with tf.Graph().as_default():
        with tf.device(tf.train.replica_device_setter(
                num_ps_tasks, merge_devices=True)):
            
            model = config.model
            model.build(config.hparams,
                        config.data_converter.output_depth,
                        is_training=True)
            # 옵티마이저
            optimizer = model.train(**_get_input_tensors(dataset_fn(), config))

            hooks = []
            if num_sync_workers:
                optimizer = tf.train.SyncReplicasOptimizer(
                    optimizer,num_sync_workers)
                hooks.append(optimizer.make_session_run_hook(is_chief))

            grads, var_list = list(zip(*optimizer.compute_gradients(model.loss)))
            global_norm = tf.global_norm(grads)
            tf.summary.scalar('global_norm', global_norm)
            
            if config.hparams.clip_mode == 'value':
                g = config.hparams.grad_clip
                clipped_grads = [tf.clip_by_value(grad, -g, g) for grad in grads]
            elif config.hparams.clip_mode == 'global_norm':
                clipped_grads = tf.cond(
                    global_norm < config.hparams.grad_norm_clip_to_zero,
                    lambda: tf.clip_by_global_norm(  # pylint:disable=g-long-lambda
                        grads, config.hparams.grad_clip, use_norm=global_norm)[0],
                    lambda: [tf.zeros(tf.shape(g)) for g in grads])
            else:
                raise ValueError(
                    'Unknown clip_mode: {}'.format(config.hparams.clip_mode))
            train_op = optimizer.apply_gradients(
                list(zip(clipped_grads, var_list)),
                global_step=model.global_step,
                name='train_step')

            logging_dict = {'global_step': model.global_step,
                            'loss': model.loss}
            
            hooks.append(tf.train.LoggingTensorHook(logging_dict, every_n_iter=100))
            if num_steps:
                hooks.append(tf.train.StopAtStepHook(last_step=num_steps))
                
            scaffold = tf.train.Scaffold(
                saver=tf.train.Saver(
                    max_to_keep=checkpoints_to_keep,
                    keep_checkpoint_every_n_hours=keep_checkpoint_every_n_hours))
            
            tf_slim.training.train(
                train_op=train_op,
                logdir=train_dir,
                scaffold=scaffold,
                hooks=hooks,
                save_checkpoint_secs=60,
                master=master,
                is_chief=is_chief)

# 학습 함수수
def run(config_map,
        tf_file_reader=tf.data.TFRecordDataset,
        file_reader=tf.python_io.tf_record_iterator,
        is_training=True):
    config = config_map['cat-drums_4bar_small']
    train_dir = './train_TFRecord'
    num_steps = 50000 # 훈련 epoch
    
    def dataset_fn():
        return data.get_dataset(
            config,
            tf_file_reader=tf_file_reader,
            is_training=True,
            cache_dataset=True)
    
    if is_training == True:
        train(
            train_dir,
            config=config,
            dataset_fn=dataset_fn,
            num_steps=num_steps)      
    
    else:
        print("EVAL")

## Run Train

In [ ]:
run(CONFIG_MAP) # epoch 50000

https://drive.google.com/file/d/1k1c8G6X-3dxltFHnc9oON_7JVEfbIs3t/view?usp=share_link  
위 링크로 체크파일 zip 다운로드 가능

# 생성

저장 될 디렉토리 생성

In [ ]:
os.mkdir("gen_midi")

magenta/magenta/models/music_vae/music_vae_generate.py 내용 일부 사용

In [ ]:
#=== magenta/magenta/models/music_vae/music_vae_generate.py ===
model = TrainedModel(
    config=CONFIG_MAP['cat-drums_4bar_small'],
    batch_size=1,
    checkpoint_dir_or_path='./train_TFRecord') # 체크포인트의 경로

generated_sequence = model.sample(n=1, length=16*4, temperature=0.5)
note_seq.sequence_proto_to_midi_file(generated_sequence[0], './gen_midi/cat-drums_4bar_small.mid')